## **Pegasus Encoder Decoder**

Imports

In [1]:
# %pip install accelerate -U
# !pip install transformers[torch]
# !pip install accelerate -U
from src.training_utils import *

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin


In [2]:
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print(transformers.__version__)

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.30.2


Load the Pubmed dataset which was downloaded to the local directory 

In [3]:
pubmed_dataset = load_dataset("ccdv/pubmed-summarization")#, data_dir ="/home/peppe/Desktop/Università/Projects/SAESum-Simultaneous_Abstractive_and_Extractive_Summarization/src/data/PubMed_abstractive")
print(pubmed_dataset)

No config specified, defaulting to: pubmed-summarization/section
Found cached dataset pubmed-summarization (/home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b)
100%|██████████| 3/3 [00:00<00:00, 1019.93it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})


Visualize some examples

In [4]:
import json
from tqdm import tqdm

def load_corpus(fname, is_training):
    corpus = []
    with open(fname, "r") as f:
        lines = f.readlines()  # Read all lines into a list
        for i, line in tqdm(enumerate(lines), total=len(lines)):
            data = json.loads(line)
            if len(data["text"]) == 0 or len(data["summary"]) == 0:
                print(i)
                continue
            if is_training:
                if len(data["indices"]) == 0 or len(data["score"]) == 0:
                    continue

            corpus.append(data)
    return corpus

val = load_corpus("src/data/PubMed/val_PUBMED.jsonl", False)
train = load_corpus("src/data/PubMed/train_PUBMED_labelled.jsonl", True)


100%|██████████| 6633/6633 [00:00<00:00, 24564.09it/s]

2320
4923


100%|██████████| 116937/116937 [00:06<00:00, 17910.40it/s]


In [5]:
pubmed_dataset['validation'][2320]

{'article': ' \n  \n  \n  \n ',
 'abstract': "according to the central limit theorem , the means of a random sample of size , n , from a population with mean ,  , and variance , 2 , distribute normally with mean ,  , and variance , 2n . using the central limit theorem , \n a variety of parametric tests have been developed under assumptions about the parameters that determine the population probability distribution . \n compared to non - parametric tests , which do not require any assumptions about the population probability distribution , parametric tests produce more accurate and precise estimates with higher statistical powers . \n however , many medical researchers use parametric tests to present their data without knowledge of the contribution of the central limit theorem to the development of such tests . \n thus , this review presents the basic concepts of the central limit theorem and its role in binomial distributions and the student 's t - test , and provides an example of the

In [6]:
pubmed_dataset['validation'][4923]

{'article': ' \n  \n  \n  \n  \n ',
 'abstract': 'highlightswe present a rare case of a congenital splenic cyst that corresponds to less than 10% of non - parasitic splenic cyst.we describe our laparoscopic approach management of a partial splenectomywe review the literature on the different treatment options for npsc based on recurrence and success rates .'}

In [7]:
from torch.utils.data import Dataset
import Levenshtein

class Clean_Dataset(Dataset):
    # This is used to clean the noisy sample from PubMed train
    def __init__(self, abs_dataset, ext_dataset, val = False):
        self.abs_dataset = abs_dataset
        self.ext_dataset = ext_dataset
        self.abs_dataset, self.ext_dataset = self.preprocess(val)


    def __len__(self):
        return len(self.ext_dataset)
    
    def __getitem__(self, idx):

        return self.abs_dataset[idx], self.ext_dataset[idx]
    
    def compare(self,s1,s2, red = True):
        distance = (s1[:len(s2)+10].strip()).startswith(s2.strip())#Levenshtein.distance(s1[:100], s2[:100])
        if not distance:
            if red:
                print(f"\nEXT: \n", s2)
                print(f"\nABS: \n", s1[:len(s2)+10])
            return False
        else:
            return True
            
    
    def preprocess(self, val = False):
        new_dataset_abs = []
        new_dataset = []
        new_ext = []
        count = 0
        for idx in tqdm(range(len(self.abs_dataset))):

            if self.abs_dataset[idx]['article'] != "":
                if not val:
                    new_dataset_abs.append({"article": self.abs_dataset[idx]['article'], 'abstract': self.abs_dataset[idx]['abstract']})
                elif val and idx not in [4923, 2320]:
                    new_dataset_abs.append({"article": self.abs_dataset[idx]['article'], 'abstract': self.abs_dataset[idx]['abstract']})

            else:
                count+=1
        if not val:
            for idx in range(len(self.ext_dataset)):
                if self.compare(new_dataset_abs[idx]['article'],self.ext_dataset[idx]['text'][0]):
                    new_dataset.append({"article": new_dataset_abs[idx]['article'], 'abstract': new_dataset_abs[idx]['abstract']})
                    new_ext.append(self.ext_dataset[idx])
                    continue
                    
                else:
                    count+=1
                    for idx2 in range(1, len(new_dataset_abs[idx:])):
                        if self.compare(new_dataset_abs[idx2+idx]['article'],self.ext_dataset[idx]['text'][0], red = False):
                            new_dataset.append({"article": new_dataset_abs[idx2+idx]['article'], 'abstract': new_dataset_abs[idx2+idx]['abstract']})
                            new_ext.append(self.ext_dataset[idx])
                            break
        else:
            new_dataset = new_dataset_abs
            new_ext = self.ext_dataset

        return new_dataset, new_ext
        

In [8]:
#train_new = Clean_Dataset(pubmed_dataset['train'], train, val = False)
val_new = Clean_Dataset(pubmed_dataset['validation'], val, val = True)
#print(len(train_new))
print(len(val_new))

100%|██████████| 6633/6633 [00:00<00:00, 17592.90it/s]

6631


In [9]:
idx = 0

val_new[idx][0]['article'], val_new[idx][1]['text']

("venous thromboembolism ( vte ) comprising of deep vein thrombosis ( dvt ) and pulmonary embolism ( pe ) can result in significant mortality , morbidity , and healthcare expenditure . \n approximately , one - third of patients with symptomatic vte manifests pe , whereas two - thirds manifest dvt alone . \n both dvt and pe can be clinically silent ( asymptomatic ) and hence not suspected . \n if undiagnosed , asymptomatic vte can lead to chronic venous disease or recurrent vte and long - term debilitating sequelae such as postthrombotic syndrome and chronic thromboembolic pulmonary hypertension . \n vte is not only disabling but also prolongs hospital stay and increases the cost of treatment . \n along with myocardial infarction and arrhythmia ( due to electrolyte imbalance ) , pe is one of the commonest causes of sudden unexplained deaths in hospitalized patients . \n it is estimated that 20 million cases of lower extremity dvt occur in the usa alone . \n the prevailing notion that th

In [10]:
serialized_data = [sample for sample in val_new]


In [11]:
with open("src/data/PubMed/Val_ExtAbs_PUBMED.json", "w") as f:
    json.dump(serialized_data, f)

In [12]:
count = 0
for idx in range(min(len(pubmed_dataset['validation']),10000000)):
    print("{} text is: {}".format(idx+1, pubmed_dataset['validation'][idx]['article'][:200]))
    # if pubmed_dataset['validation'][idx]['article'] == "":
    #     count += 1 
        # print(pubmed_dataset['train'][idx])
print(count)

1 text is: venous thromboembolism ( vte ) comprising of deep vein thrombosis ( dvt ) and pulmonary embolism ( pe ) can result in significant mortality , morbidity , and healthcare expenditure . 
 approximately ,
2 text is: there is an epidemic of stroke in low and middle - income countries due to rapidly increasing prevalence of vascular risk factors such as hypertension , diabetes mellitus , dyslipidaemia . 
 the need 
3 text is: cardiovascular diseases account for the highest mortality rate worldwide and are expected to be the major cause of death by 2020 ( 1 ) . 
 studies in iran showed that cardiovascular diseases are the m
4 text is: results of a liquid culturing system ( bd bactec mgit 960 system ; bd diagnostics , sparks , md , us ) ( for details , see the technical appendix ) showed that all 5 beijing and 5 eai genotype strains
5 text is: the need for magnetic resonance imaging ( mri ) in patients with an implanted pacemaker or implantable cardioverter - defibrillator ( icd ) i

Load tokenizer and model

In [13]:
model_checkpoint = "google/pegasus-x-base" # Use pegasus-x-base-finetuned-xsum
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(tokenizer)
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
print(model)

PegasusTokenizerFast(name_or_path='google/pegasus-x-base', vocab_size=96103, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', '<

Load the metric

In [14]:
from evaluate import load
metric = load("rouge")
print(metric)

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

Create pre-processing function

In [15]:
max_input_length = 512
max_target_length = 256
#DA MODIFICARFE PER PEGASUS FORSE
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["abstract"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
preprocess_function(pubmed_dataset['train'][:1])

{'input_ids': [[114, 909, 11624, 1382, 2375, 120, 115, 2651, 110, 108, 47422, 143, 110, 38117, 233, 57468, 110, 158, 604, 404, 3629, 197, 371, 231, 195, 26364, 110, 108, 21472, 132, 11526, 70443, 111, 56744, 143, 13076, 233, 57424, 110, 158, 604, 195, 26364, 110, 108, 21472, 132, 11526, 72225, 115, 109, 1690, 1105, 110, 107, 115, 110, 35057, 114, 692, 790, 110, 58373, 281, 399, 2220, 115, 110, 116, 43449, 111, 35739, 33824, 19244, 2375, 19107, 113, 305, 76040, 110, 108, 110, 74760, 111, 48254, 110, 108, 118, 72225, 110, 108, 15713, 111, 10703, 110, 108, 4802, 110, 107, 109, 19107, 113, 40999, 790, 8225, 399, 4843, 404, 115, 40727, 8032, 6005, 135, 20177, 112, 30316, 110, 107, 53018, 21026, 692, 113, 8225, 399, 392, 115, 32043, 28064, 3264, 120, 30316, 113, 183, 4512, 135, 40999, 111, 580, 513, 1050, 110, 107, 6406, 246, 133, 3251, 233, 4087, 79876, 644, 111, 256, 319, 371, 233, 377, 3957, 113, 3085, 943, 242, 110, 107, 9939, 110, 108, 399, 5173, 962, 127, 850, 130, 109, 1146, 962, 110,

## **TRAINING**

In [16]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [17]:
tokenized_datasets = pubmed_dataset.map(preprocess_function, batched=True)
print(tokenized_datasets["train"][0]["article"])

Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-83fe1a3796a65436.arrow
Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-251fe14d67349aa9.arrow
Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-33ed9b12aba01fbd.arrow


a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries . 
 in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively . 
 the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% . 
 anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight . 
 snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states 
 there are also some reports regarding school feeding programs in developing countries . in 

In [18]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    "Pegasus-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate = True,
    fp16=True,
    gradient_accumulation_steps=512,
    logging_steps=1,
    label_smoothing_factor = 0.1, 
    auto_find_batch_size = True,
)

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    wandb.log({'rouge1': result['rouge1'], 'rouge2': result['rouge2'], 'rougeL': result['rougeL'], 'rougeLsum': result['rougeLsum']})
    

    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [20]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)
for param in trainer.model.model.encoder.parameters():
    param.requires_grad = False

In [21]:
trainer.train()
wandb.finish()

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/29 [00:00<?, ?it/s]You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2 wandb.finish()                                                                               │
│   3                                                                                              │
│                                                                                                  │
│ /home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/transformers/trainer.py:1645 in   │
│ train                                                                                            │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/accelerate/utils/memory.py:132 in │
│ decorator                                                                                        │
│                                                                                                  │
│   129 │   │   │   if batch_size == 0:                                                            │
│   130 │   │   │   │   raise RuntimeError("No executable batch size found, reached zero.")        │
│   131 │   │   │   try:                                                                           │
│ ❱ 132 │   │   │   │   return function(batch_size, *args, **kwargs)                               │
│   133 │   │   │   except Exception as e:                                                         │
│   134 │   │   │   │   if should_reduce_batch_size(e):                                            │
│   135 │   │   │   │   │   gc.collect()                                                           │
│                                                                                                  │
│ /home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/transformers/trainer.py:1938 in   │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                            

In [ ]:
model_checkpoint = "checkpoint-14500-finetuned_alot/checkpoint-29500"

Already fine-tuned on pubmed

In [ ]:
model_checkpoint = "google/pegasus-pubmed"

In [ ]:
model_checkpoint = "Kevincp560/pegasus-arxiv-finetuned-pubmed"

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# import pegasus
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-pubmed")


ARTICLE_TO_SUMMARIZE = "Researchers at a leading university have developed a groundbreaking technology that could revolutionize renewable energy generation. The new system, known as 'SolarWave,' harnesses the power of ocean waves to generate electricity. By utilizing a network of specialized buoys equipped with advanced turbines, the technology can convert the kinetic energy from the waves into clean, sustainable power. This innovation has the potential to significantly contribute to the global efforts in combating climate change and reducing our reliance on fossil fuels. It's an exciting development that could reshape the future of renewable energy."
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt", truncation=True)

# Generate Summary
summary_ids = model.generate(inputs["input_ids"].to(device))
output = tokenizer.batch_decode(summary_ids, skip_special_tokens=True,
                                clean_up_tokenization_spaces=False)[0]
print(output)

print(len(output))